# 第15章 奇异值分解

#### 矩阵的奇异值分解是指将 $m \times n$ 实矩阵 $A$ 表示为以下三个实矩阵乘积形式的运算
$$A = U \Sigma V ^ { T }$$
其中 $U$ 是 $m$ 阶正交矩阵，$V$ 是 $n$ 阶正交矩阵，$\Sigma$ 是 $m \times n$ 矩形对角矩阵
$$\Sigma = \operatorname { diag } ( \sigma _ { 1 } , \sigma _ { 2 } , \cdots , \sigma _ { p } ) , \quad p = \operatorname { min } \{ m , n \}$$
其对角线元素非负，且满足$\sigma _ { 1 } \geq \sigma _ { 2 } \geq \cdots \geq \sigma _ { p } \geq 0$

**紧奇奇异值分解**是与原始矩阵等秩的奇异值分解

**截断奇异值分解**是比原始矩阵低秩的奇异值分解。

In [4]:
# 实现奇异值分解， 输入一个numpy矩阵，输出 U, sigma, V
import numpy as np

#基于矩阵分解的结果，复原矩阵
def rebuildMatrix(U, sigma, V):
  a = np.dot(U, sigma)
  a = np.dot(a, np.transpose(V))
  return a

#基于特征值的大小，对特征值以及特征向量进行排序。倒序排列
def sortByEigenValue(Eigenvalues, EigenVectors):
  index = np.argsort(-1 * Eigenvalues)
  Eigenvalues = Eigenvalues[index]
  EigenVectors = EigenVectors[:, index]
  return Eigenvalues, EigenVectors

#对一个矩阵进行奇异值分解
def SVD(matrixA, NumOfLeft=None):
  #NumOfLeft是要保留的奇异值的个数，也就是中间那个方阵的宽度
  #首先求transpose(A)*A
  matrixAT_matrixA = np.dot(np.transpose(matrixA), matrixA)
  #然后求右奇异向量
  lambda_V, X_V = np.linalg.eig(matrixAT_matrixA)
  lambda_V, X_V = sortByEigenValue(lambda_V, X_V)
  #求奇异值
  sigmas = lambda_V
  sigmas = list(map(lambda x: np.sqrt(x) if x > 0 else 0, sigmas))  #python里很小的数有时候是负数
  sigmas = np.array(sigmas)
  sigmasMatrix = np.diag(sigmas)
  if NumOfLeft == None:
    rankOfSigmasMatrix = len(list(filter(lambda x: x > 0, sigmas)))  #大于0的特征值的个数
  else:
    rankOfSigmasMatrix = NumOfLeft
  sigmasMatrix = sigmasMatrix[0:rankOfSigmasMatrix, :]  #特征值为0的奇异值就不要了

  #计算右奇异向量
  X_U = np.zeros((matrixA.shape[0], rankOfSigmasMatrix))  #初始化一个右奇异向量矩阵，这里直接进行裁剪
  for i in range(rankOfSigmasMatrix):
      X_U[:, i] = np.transpose(np.dot(matrixA, X_V[:, i]) / sigmas[i])

  #对右奇异向量和奇异值矩阵进行裁剪
  X_V = X_V[:, 0:NumOfLeft]
  sigmasMatrix = sigmasMatrix[0:rankOfSigmasMatrix, 0:rankOfSigmasMatrix]

  return X_U, sigmasMatrix, X_V

# ----------------------------------
# TEST
A = np.array([[1, 1, 1, 2, 2], [0, 0, 0, 3, 3], [0, 0, 0, 1, 1], [1, 1, 1, 0, 0],
              [2, 2, 2, 0, 0], [5, 5, 5, 0, 0], [1, 1, 1, 0, 0]])
print("The shape of matrix: {}".format(A.shape))

print("-----------------------------------------")
X_U, sigmasMatrix, X_V = SVD(A, NumOfLeft=3)
print("The U of SVD matrix decomposition:\n{}".format(X_U))
print("-----------------------------------------")
print("The V of SVD matrix decomposition:\n{}".format(X_V))
print("-----------------------------------------")
print("The sigmas of SVD matrix decomposition:\n{}".format(sigmasMatrix))
print("-----------------------------------------")
# rebuild from U, sigma, V
rebuild_matrix = rebuildMatrix(X_U, sigmasMatrix, X_V)
print("The re-build matrix:\n{}".format(rebuild_matrix))
print("-----------------------------------------")
print("The original matrix:\n{}".format(A))
print("-----------------------------------------")

The shape of matrix: (7, 5)
-----------------------------------------
The U of SVD matrix decomposition:
[[ 1.96602638e-01 -5.12980706e-01 -1.01842345e-08]
 [ 3.08997616e-02 -8.04794293e-01  3.45625212e-09]
 [ 1.02999205e-02 -2.68264764e-01  1.15208404e-09]
 [ 1.76002797e-01  2.35488225e-02 -1.22083430e-08]
 [ 3.52005594e-01  4.70976451e-02 -2.44166860e-08]
 [ 8.80013984e-01  1.17744113e-01 -5.58095680e-08]
 [ 1.76002797e-01  2.35488225e-02 -1.22083430e-08]]
-----------------------------------------
The V of SVD matrix decomposition:
[[ 5.75872999e-01  4.12749590e-02  8.16496581e-01]
 [ 5.75872999e-01  4.12749590e-02 -4.08248290e-01]
 [ 5.75872999e-01  4.12749590e-02 -4.08248290e-01]
 [ 5.05512944e-02 -7.05297502e-01  3.66695611e-17]
 [ 5.05512944e-02 -7.05297502e-01  3.66695611e-17]]
-----------------------------------------
The sigmas of SVD matrix decomposition:
[[9.81586105e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.25821946e+00 0.00000000e+00]
 [0.00000000e+00 0.0000000

### 矩阵的奇异值分解，结合 PCA，可以对|数据中最重要的特征进行排序，进行压缩数据
### 特别对于图像数据而言